In [ ]:
import pandas as pd
import numpy as np
from PIL import Image
import os

def rle2mask(mask_rle, shape):
    '''
    Функция для преобразования RLE маски в 2D numpy массив.
    :param mask_rle: run-length as string formatted (start length)
    :param shape: (height, width) of array to return
    Returns numpy array, 1 - mask, 0 - background
    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape, order='F')

# Загрузка метаданных
metadata = pd.read_csv('/kaggle/input/sartorius-cell-instance-segmentation/train.csv')

# Загрузка изображений и масок
data_path = '/kaggle/input/sartorius-cell-instance-segmentation/train'
images = []  # список для хранения изображений
masks = []   # список для хранения масок

for file_name in os.listdir(data_path):
    # Загрузка изображения
    img = Image.open(os.path.join(data_path, file_name))
    img = img.resize((416, 416))  # Изменение размера изображения для совместимости с YOLO
    images.append(np.array(img))
    
    # Загрузка соответствующей маски
    img_id = file_name.split('.')[0]  # Идентификатор изображения
    mask_rle = metadata.loc[metadata['id'] == img_id, 'annotation'].values[0]  # RLE маска
    mask = rle2mask(mask_rle, img.size)
    masks.append(mask)



In [ ]:
import cv2

for img in images:
    # Подготовка изображения
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)  # Переход от RGB к BGR, т.к. OpenCV использует BGR
    img = img / 255.0  # Нормализация изображения
    img = torch.from_numpy(img).permute(2, 0, 1).float().to('cuda' if torch.cuda.is_available() else 'cpu')  # Переупорядочивание осей и перевод изображения в тензор PyTorch
    img = img.unsqueeze(0)  # Добавление размерности для пакета

    with torch.no_grad():
        # Прямой проход (forward pass)
        outputs = model(img)

  


In [ ]:
import os

# Путь к директории
#directory = '/kaggle/working'

# Рекурсивно удалить все файлы в директории и поддиректориях
#for root, dirs, files in os.walk(directory):
#    for file_name in files:
#        file_path = os.path.join(root, file_name)
#os.remove(file_path)

!pip install torchvision yolov5        
import subprocess
packages = [
'aiobotocore<1.29.77,>=1.29.76',
'jupyter-lsp>=2.0.0',
'shapely>=2',
'packaging>=22.0',
'scipy<1.10,>=1.4.1'
]
for package in packages:
    subprocess.call(['pip', 'install', package])


In [ ]:
import os
import glob
import json
import shutil
impoty yaml

def process_annotations(json_path, annotations_path):
    with open(json_path) as f:
        data = json.load(f)

    temp_annotations_path = os.path.join(annotations_path, 'temp')
    os.makedirs(temp_annotations_path, exist_ok=True)

    for annotation_id, annotation_data in data['annotations'].items():
        image_id = annotation_data['image_id']
        category_id = annotation_data['category_id'] - 1
        bbox = annotation_data['bbox']
        xmin, ymin, width, height = bbox

        image_data = next(image for image in data['images'] if image['id'] == image_id)
        img_width, img_height = image_data['width'], image_data['height']

        x_center = (xmin + width / 2) / img_width
        y_center = (ymin + height / 2) / img_height
        bbox_width = width / img_width
        bbox_height = height / img_height

        annotation_file = os.path.join(temp_annotations_path, f'{annotation_id}.txt')
        with open(annotation_file, 'a') as f:
            f.write(f'{category_id} {x_center} {y_center} {bbox_width} {bbox_height}\n')

    move_annotations(temp_annotations_path, annotations_path)
    os.rmdir(temp_annotations_path)

def move_annotations(source_folder, destination_folder):
    os.makedirs(destination_folder, exist_ok=True)

    for file_name in os.listdir(source_folder):
        source_path = os.path.join(source_folder, file_name)
        destination_path = os.path.join(destination_folder, file_name)

        if os.path.exists(destination_path):
            os.remove(destination_path)

        shutil.move(source_path, destination_path)

def create_data_files(image_dir, output_train_file, output_val_file, train_ratio=0.8):
    image_files = glob.glob(os.path.join(image_dir, '*.png'))
    num_images = len(image_files)
    num_train = int(train_ratio * num_images)
    train_files = image_files[:num_train]
    val_files = image_files[num_train:]

    with open(output_train_file, 'w') as f:
        for path in train_files:
            f.write(path + '\n')

    if output_val_file:
        with open(output_val_file, 'w') as f:
            for path in val_files:
                f.write(path + '\n')

# Путь к папке LIVECell_dataset_2021
dataset_folder = '/kaggle/input/sartorius-cell-instance-segmentation/LIVECell_dataset_2021'

# Путь к папке, где будут храниться аннотации в формате YOLO
yolo_annotations_path = '/kaggle/temp/data/annotations'

# Обработка аннотаций для папки LIVECell
livecell_annotations_folder = os.path.join(dataset_folder, 'annotations/LIVECell')
process_annotations(os.path.join(livecell_annotations_folder, 'livecell_coco_test.json'), yolo_annotations_path)
process_annotations(os.path.join(livecell_annotations_folder, 'livecell_coco_train.json'), yolo_annotations_path)
process_annotations(os.path.join(livecell_annotations_folder, 'livecell_coco_val.json'), yolo_annotations_path)

# Обработка аннотаций для папки LiveCell_single_cells
livecell_single_cells_annotations_folder = os.path.join(dataset_folder, 'annotations/LIVECell_single_cells')

for category_folder in os.listdir(livecell_single_cells_annotations_folder):
    category_folder_path = os.path.join(livecell_single_cells_annotations_folder, category_folder)
    
    if os.path.isdir(category_folder_path):
        test_file = f'livecell_{category_folder}_test.json'
        train_file = f'livecell_{category_folder}_train.json'
        val_file = f'livecell_{category_folder}_val.json'
        
        process_annotations(os.path.join(category_folder_path, test_file), yolo_annotations_path)
        process_annotations(os.path.join(category_folder_path, train_file), yolo_annotations_path)
        process_annotations(os.path.join(category_folder_path, val_file), yolo_annotations_path)

# Путь к папке с изображениями для train.txt и val.txt
livecell_train_val_images_folder = os.path.join(dataset_folder, 'images/livecell_train_val_images')

# Создание train.txt и val.txt
create_data_files(livecell_train_val_images_folder, '/kaggle/temp/data/annotations/train.txt',
                  '/kaggle/temp/data/annotations/val.txt', train_ratio=0.8)

# Путь к папке с изображениями для test.txt
livecell_test_images_folder = os.path.join(dataset_folder, 'images/livecell_test_images')

# Создание test.txt
create_data_files(livecell_test_images_folder, '/kaggle/temp/data/annotations/test.txt', '')

# Путь к папке с изображениями для test.txt (дополнительная папка test)
test_images_folder = '/kaggle/input/sartorius-cell-instance-segmentation/test'

# Создание test.txt для дополнительной папки test
create_data_files(test_images_folder, '/kaggle/temp/data/annotations/test.txt', '')



In [ ]:
import os
import yaml
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor, Lambda, Compose
import torch
import matplotlib.pyplot as plt

# Путь к папке для сохранения YAML-файла
output_folder = '/kaggle/temp/data'
os.makedirs(output_folder, exist_ok=True)

# The structure of the data for YAML
data = {
    'nc': 2,  # Количество классов
    'names': ['neuron', 'background'], # Имена классов
    'depth_multiple': 0.33,
    'width_multiple':0.50,
    'anchors': [
        [10, 13, 16, 30, 33, 23],
        [30, 61, 62, 45, 59, 119],
        [116, 90, 156, 198, 373, 326]
    ],
    'backbone': [
        [-1, 1, 'Focus', [64, 3]],
        [-1, 1, 'Conv', [128, 3, 2]],
        [-1, 3, 'BottleneckCSP', [128]],
        [-1, 1, 'Conv', [256, 3, 2]],
        [-1, 9, 'BottleneckCSP', [256]],
        [-1, 1, 'Conv', [512, 3, 2]],
        [-1, 9, 'BottleneckCSP', [512]],
        [-1, 1, 'Conv', [1024, 3, 2]],
        [-1, 1, 'SPP', [1024, [5, 9, 13]]],
        [-1, 3, 'BottleneckCSP', [1024, False]]
    ],
    'head': [
        [-1, 1, 'Conv', [512, 1, 1]],
        [-1, 1, 'nn.Upsample', [None, 2, 'nearest']],
        [[-1, 6], 1, 'Concat', [1]],
        [-1, 3, 'BottleneckCSP', [512, False]],
        [-1, 1, 'Conv', [256, 1, 1]],
        [-1, 1, 'nn.Upsample', [None, 2, 'nearest']],
        [[-1, 4], 1, 'Concat', [1]],
        [-1, 3, 'BottleneckCSP', [256, False]],
        [-1, 1, 'Conv', [256, 3, 2]],
        [[-1, 14], 1, 'Concat', [1]],
        [-1, 3, 'BottleneckCSP', [512, False]],
        [-1, 1, 'Conv', [512, 3, 2]],
        [[-1, 10], 1, 'Concat', [1]],
        [-1, 3, 'BottleneckCSP', [1024, False]],
        [[17, 20, 23], 1, 'Detect', [2, [[10, 13, 16, 30, 33, 23], [30, 61, 62, 45, 59, 119], [116, 90, 156, 198, 373, 326]]]]
    ],
    'train': '/kaggle/temp/data/annotations/train.txt',  # Путь к файлу train.txt
    'val': '/kaggle/temp/data/annotations/val.txt',  # Путь к файлу val.txt
    'test': '/kaggle/temp/data/annotations/test.txt'  # Путь к файлу test.txt
}



# Путь к файлу YAML
yaml_file_path = os.path.join(output_folder, 'yaml_file.yaml')

# Запись словаря в файл YAML
with open(yaml_file_path, 'w') as f:
    yaml.dump(data, f)

In [ ]:
import torch
# Клонирование репозитория YOLOv5
!git clone https://github.com/ultralytics/yolov5.git

# Переход в папку YOLOv5
os.chdir('yolov5')
!pip install -r requirements.txt  # install
model = torch.hub.load("ultralytics/yolov5", "yolov5n")

In [ ]:
import os
from yolov5 import train
import cv2
import pandas as pd
import requests

def download_weights(url, save_path):
    response = requests.get(url, stream=True)
    with open(save_path, 'wb') as f:
        f.write(response.content)
# URL для загрузки весов модели YOLOv5
url = 'https://github.com/ultralytics/yolov5/releases/download/v5.0/yolov5n.pt'
save_path = '/kaggle/working/yourweights.pth'
# Загрузка весов YOLOv5 с GitHub
weights = download_weights(url, save_path)
#weights = 'yolov5s.pt'  # Путь к предварительно обученным весам модели


In [ ]:
import torch
import os

!git clone https://github.com/ultralytics/yolov5.git

# Переход в папку YOLOv5
os.chdir('yolov5')
!pip install -r requirements.txt  # install

model = torch.hub.load("ultralytics/yolov5", "yolov5s")

# Путь к файлу yaml с конфигурацией обучения
data_yaml = '/kaggle/temp/data/yaml_file.yaml'

In [ ]:
# Настройки обучения
batch_size = 16
epochs = 10

import torch
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR


from yolov5.models.yolo import Model
from yolov5.utils.dataloaders import LoadImagesAndLabels
from yolov5.utils.loss import ComputeLoss

# Путь к файлу yaml с конфигурацией обучения
#data_yaml = '/kaggle/temp/data/yaml_file.yaml'

# Загрузка данных для обучения
dataset = LoadImagesAndLabels(livecell_train_val_images_folder)

# Создание DataLoader для обучения
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)

# Определение модели
model = Model('/kaggle/temp/data/yaml_file.yaml')

# Определение функции потерь
criterion = ComputeLoss

# Определение оптимизатора
learning_rate = 0.001
optimizer = Adam(model.parameters(), lr=learning_rate)

# Определение шедулера для изменения learning rate
lr_step_size = 5
lr_gamma = 0.1
scheduler = StepLR(optimizer, step_size=lr_step_size, gamma=lr_gamma)

# Перемещение модели на GPU (если доступен)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Цикл обучения
for epoch in range(epochs):
    model.train()
    total_loss = 0.0
    total_samples = 0
    log_interval = 10

    for batch_idx, (images, targets) in enumerate(dataloader):
        images = images.to(device)
        targets = targets.to(device)
        
        # Обнуление градиентов
        optimizer.zero_grad()
        
        # Прямой проход через модель
        outputs = model.forward(images)
        
        # Расчет функции потерь
        loss, loss_items = criterion(outputs, targets)

        # Обратный проход и обновление весов
        loss.backward()
        optimizer.step()
        
        # Суммарная потеря во время эпохи
        total_loss += loss.item()
        total_samples += targets.shape[0]
        
        # Вывод информации о процессе обучения
        if batch_idx % log_interval == 0:
            print(f"Epoch {epoch+1}/{epochs}, Batch {batch_idx}/{len(dataloader)}, Loss: {loss.item():.4f}")
    
    # Печать средней потери в конце эпохи
    avg_loss = total_loss / len(dataloader)
    print(f"Epoch {epoch+1}/{epochs}, Average Loss: {avg_loss:.4f}")
    
    # Обновление learning rate
    scheduler.step()

# Сохранение обученной модели
# torch.save(model.state_dict(), saved_model_path)



In [ ]:
import torch
from torch.utils.data import DataLoader
from torchvision.transforms import functional as F
from torchvision.models.detection import maskrcnn_resnet50_fpn

# Загрузка и подготовка данных для обучения Mask R-CNN

# Загрузка аннотаций и изображений
annotations = load_annotations()  # Функция загрузки аннотаций
images = load_images()  # Функция загрузки изображений

# Создание датасета и даталоадера для Mask R-CNN
dataset = MyCustomDataset(annotations, images)  # Замените на свой класс датасета
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Создание модели Mask R-CNN
model = maskrcnn_resnet50_fpn(pretrained=True)

# Определение оптимизатора и функции потерь для обучения Mask R-CNN
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=0.0005)
criterion = MaskRCNNLoss()

# Обучение Mask R-CNN
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    
    for images, targets in dataloader:
        images = [F.to_tensor(image) for image in images]
        targets = [{k: v for k, v in t.items()} for t in targets]
        
        # Передача данных на GPU (если доступно)
        images = [image.to(device) for image in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        
        # Обновление весов модели
        optimizer.zero_grad()
        loss_dict = model(images, targets)
        loss = sum(loss for loss in loss_dict.values())
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    # Вывод прогресса обучения
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(dataloader)}")

# Сохранение обученной модели Mask R-CNN
torch.save(model.state_dict(), "mask_rcnn_model.pth")



In [ ]:
model.eval()  # Переводим модель в режим оценки (валидации или тестирования)

# Валидация
with torch.no_grad():
    total_loss = 0.0
    total_iou = 0.0
    
    for images, targets in validation_dataloader:  # Замените на свой даталоадер валидации
        images = [F.to_tensor(image) for image in images]
        targets = [{k: v for k, v in t.items()} for t in targets]
        
        # Передача данных на GPU (если доступно)
        images = [image.to(device) for image in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        
        # Вычисление предсказаний модели
        predictions = model(images)
        
        # Вычисление функции потерь
        loss_dict = model(images, targets)
        loss = sum(loss for loss in loss_dict.values())
        
        # Вычисление метрики IoU
        iou = calculate_iou(targets, predictions)
        
        total_loss += loss.item()
        total_iou += iou
    
    avg_loss = total_loss / len(validation_dataloader)
    avg_iou = total_iou / len(validation_dataloader)
    
    print(f"Validation Loss: {avg_loss}, IoU: {avg_iou}")

# Тестирование
with torch.no_grad():
    total_iou = 0.0
    
    for images, targets in test_dataloader:  
        images = [F.to_tensor(image) for image in images]
        targets = [{k: v for k, v in t.items()} for t in targets]
        
        # Передача данных на GPU (если доступно)
        images = [image.to(device) for image in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        
        # Вычисление предсказаний модели
        predictions = model(images)
        
        # Вычисление метрики IoU
        iou = calculate_iou(targets, predictions)
        
        total_iou += iou
    
    avg_iou = total_iou / len(test_dataloader)
    
    print(f"Test IoU: {avg_iou}")

